In [ ]:
import pathlib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
from orchid import (project_loader as opl)

In [ ]:
import toolz.curried as toolz

In [ ]:
import UnitsNet

In [ ]:
project_filenames = {
    'permian-a': 'Project-frankNstein_Permian_UTM13FT_0412_PjtDataFrame.ifrac',
    # 'gng': 'GnG_DemoProject_wDataFrames.ifrac',
}

In [ ]:
test_data_path = pathlib.Path('c:/src/Orchid.IntegrationTestData/')
project_path_names = toolz.valmap(lambda fn: test_data_path.joinpath(fn), project_filenames)
project_path_names

In [ ]:
projects = toolz.valmap(
    lambda pn: opl.ProjectLoader(str(pn)).native_project(),
    project_path_names)
projects

In [ ]:
project_units = toolz.valmap(lambda p: p.ProjectUnits, projects)
project_units

In [ ]:
net_data_frames = projects['permian-a'].DataFrames.Items